In [1]:
import os
import glob
import numpy as np
import time
from params import par
from torchvision import transforms
from PIL import Image
import torch
import math
from helper import R_to_angle

def clean_unused_images():
    seq_frame = {'00': ['000', '004540'],
                '01': ['000', '001100'],
                '02': ['000', '004660'],
                '03': ['000', '000800'],
                '04': ['000', '000270'],
                '05': ['000', '002760'],
                '06': ['000', '001100'],
                '07': ['000', '001100'],
                '08': ['001100', '005170'],
                '09': ['000', '001590'],
                '10': ['000', '001200']
                }
    for dir_id, img_ids in seq_frame.items():
        dir_path = '{}{}/'.format(par.image_dir, dir_id)
        if not os.path.exists(dir_path):
            continue

        print('Cleaning {} directory'.format(dir_id))
        start, end = img_ids
        start, end = int(start), int(end)
        for idx in range(0, start):
            img_name = '{:010d}.png'.format(idx)
            img_path = '{}{}/{}'.format(par.image_dir, dir_id, img_name)
            if os.path.isfile(img_path):
                os.remove(img_path)
        for idx in range(end+1, 10000):
            img_name = '{:010d}.png'.format(idx)
            img_path = '{}{}/{}'.format(par.image_dir, dir_id, img_name)
            if os.path.isfile(img_path):
                os.remove(img_path)
                
        clean_times = []
        original_times = []
        with open(f'{dir_path}/times.txt', 'r') as f:
            for i, line in enumerate(f.readlines()):
                if i >= start and i <= end:
                    clean_times.append(line)
        with open(f'{dir_path}/clean_times.txt', 'w') as f:
            for line in clean_times:
                f.write(line)
            


# transform poseGT [R|t] to [theta_x, theta_y, theta_z, x, y, z]
# save as .npy file
def create_pose_data():
    info = {'00': [0, 4540], '01': [0, 1100], '02': [0, 4660], '03': [0, 800], '04': [0, 270], '05': [0, 2760], '06': [0, 1100], '07': [0, 1100], '08': [1100, 5170], '09': [0, 1590], '10': [0, 1200]}
    start_t = time.time()
    for video in info.keys():
        fn = '{}{}.txt'.format(par.pose_dir, video)
        print('Transforming {}...'.format(fn))
        with open(fn) as f:
            lines = [line.split('\n')[0] for line in f.readlines()]
            poses = [R_to_angle([float(value) for value in l.split(' ')]) for l in lines]  # list of pose (pose=list of 12 floats)
            poses = np.array(poses)
            base_fn = os.path.splitext(fn)[0]
            np.save(base_fn+'.npy', poses)
            print('Video {}: shape={}'.format(video, poses.shape))
    print('elapsed time = {}'.format(time.time()-start_t))


def calculate_bw_mean_std(image_path_list, minus_point_5=False):
    n_images = len(image_path_list)
    cnt_pixels = 0
    print('Numbers of frames in training dataset: {}'.format(n_images))
    mean_np = 0.0
    mean_tensor = 0.0
    to_tensor = transforms.ToTensor()

    image_sequence = []
    for idx, img_path in enumerate(image_path_list):
        print('{} / {}'.format(idx, n_images), end='\r')
        img_as_img = Image.open(img_path)
        img_as_tensor = to_tensor(img_as_img)
        if minus_point_5:
            img_as_tensor = img_as_tensor - 0.5
        img_as_np = np.array(img_as_img)
        # img_as_np = np.rollaxis(img_as_np, 2, 0)
        cnt_pixels += img_as_np.shape[0]*img_as_np.shape[1]

        mean_tensor += float(torch.sum(img_as_tensor))
        mean_np += float(np.sum(img_as_np))

    mean_tensor = mean_tensor / cnt_pixels
    mean_np = mean_np / cnt_pixels
    print('mean_tensor = ', mean_tensor)
    print('mean_np = ', mean_np)

    std_tensor = 0.0
    std_np = 0.0
    for idx, img_path in enumerate(image_path_list):
        print('{} / {}'.format(idx, n_images), end='\r')
        img_as_img = Image.open(img_path)
        img_as_tensor = to_tensor(img_as_img)
        if minus_point_5:
            img_as_tensor = img_as_tensor - 0.5
        img_as_np = np.array(img_as_img)
        # img_as_np = np.rollaxis(img_as_np, 2, 0)

        tmp = (img_as_tensor - mean_tensor)**2
        std_tensor += float(torch.sum(tmp))
        tmp = (img_as_np - mean_np)**2
        std_np += float(np.sum(tmp))

    std_tensor = math.sqrt(std_tensor / cnt_pixels)
    std_np = math.sqrt(std_np / cnt_pixels)
    print('std_tensor = ', std_tensor)
    print('std_np = ', std_np)


clean_unused_images()
run = False
if run:
    create_pose_data()

    # Calculate Black & White means of images in training videos
    train_video = ['00', '02', '08', '09', '06', '04', '10']
    image_path_list = []
    for folder in train_video:
        image_path_list += glob.glob('dataset/dataset/sequences/{}/image_0/*.png'.format(folder))
    calculate_bw_mean_std(image_path_list, minus_point_5=True)



c:\users\vilad\appdata\local\programs\python\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cleaning 00 directory
Cleaning 01 directory
Cleaning 02 directory
Cleaning 03 directory
Cleaning 04 directory
Cleaning 05 directory
Cleaning 06 directory
Cleaning 07 directory
Cleaning 08 directory
Cleaning 09 directory
Cleaning 10 directory
